In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math

Greenwich uc Time 
Time shifts - not on weather data though 
Realized supply Germany 
Installed capacity 
Total wind is what we care about - other stuff is not really relevant to the task 
Install capacity - 1 mega Watt large to be included, turbines that are installed could produce 
What is autoregressive ? 

We should be able to forecas one hour ahead 
U and v component are relevant - if you calculate euc distance between u and v 10 you get the speed 
Ssr - solar is important 
Is the method we used a hood method for the task ? - discussion, was the decision I made during the course good

Neutrally present results, than opinion
Interpretation on why something is the case, leave it to the discussion 
Boil down, what is the core message you want the reader to get
Guide the reader through the paper 

In [2]:
# read multiple csv from data folder
files = []
for file in os.listdir('data'):
    if file.endswith('.csv'):
        files.append(file)
print(files)

['Weather_Data_Germany.csv', 'Prices_Europe.csv', 'Realised_Supply_Germany.csv', 'Installed_Capacity_Germany.csv', 'Reaslised_Demand_Germany.csv']


In [3]:
df = pd.read_csv('data/' + files[0])

In [4]:
df

,longitude,latitude,forecast_origin,time,cdir,z,msl,blh,tcc,u10,v10,t2m,ssr,tsr,sund,tp,fsr,u100,v100
0,5.8,54.2,2019-01-01 00:00:00,2019-01-01 00:00:00,0.0,-2.766602,102439.914,766.03015,0.304399,11.806978,2.823900,282.04110,0.0,0.0,0.000,0.000000,0.000860,15.666069,3.153420
1,5.8,53.2,2019-01-01 00:00:00,2019-01-01 00:00:00,0.0,-6.159180,102795.230,842.93190,0.945699,6.628730,1.767059,281.28223,0.0,0.0,0.000,0.000000,0.254992,10.824787,2.401394
2,5.8,52.2,2019-01-01 00:00:00,2019-01-01 00:00:00,0.0,478.410160,103077.450,862.93030,0.999504,4.976940,1.899902,280.48530,0.0,0.0,0.000,0.000000,0.439765,8.450895,2.586425
3,5.8,51.2,2019-01-01 00:00:00,2019-01-01 00:00:00,0.0,237.821290,103310.510,619.56150,0.999992,3.810944,1.201527,280.26593,0.0,0.0,0.000,0.000000,0.645616,6.853557,1.614200
4,5.8,50.2,2019-01-01 00:00:00,2019-01-01 00:00:00,0.0,4681.780000,103456.680,385.46838,0.999992,3.689941,-0.424220,277.94235,0.0,0.0,0.000,0.000000,0.361133,6.055728,-0.896704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104315,14.8,51.2,2021-12-31 00:00:00,2021-12-31 23:00:00,2835274.0,2180.339400,101831.690,838.02860,1.000008,6.772395,-0.635157,283.54050,2171042.0,4075276.0,10313.742,0.001773,0.680833,11.236713,-1.493503
2104316,14.8,50.2,2021-12-31 00:00:00,2021-12-31 23:00:00,3174462.0,1949.363300,102003.055,815.65460,0.981605,5.854994,0.206125,284.05286,3600288.0,5727868.0,25293.312,0.000539,0.622661,10.005124,-0.225957
2104317,14.8,49.2,2021-12-31 00:00:00,2021-12-31 23:00:00,3614996.0,4384.649000,102251.730,741.03357,0.630130,5.271448,-1.014326,281.04538,3281948.0,5547652.0,25715.790,0.002406,0.851062,9.639029,-2.366759
2104318,14.8,48.2,2021-12-31 00:00:00,2021-12-31 23:00:00,3894518.0,3073.145500,102469.890,420.26746,0.440941,4.102491,-0.592839,280.20236,4438841.0,6871398.0,27216.710,0.000576,0.897962,8.555898,-1.410386


In [ ]:
df.shape

In [ ]:
plt.scatter(x=df['longitude'], y=df['latitude'])
plt.show()

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
path_to_germany = "./data/vg2500_geo84/vg2500_bld.shp"
germany_gdf = gpd.read_file(path_to_germany)

In [ ]:
germany_gdf.plot()

In [ ]:
print("CRS for Germany shapefile:", germany_gdf.crs)

In [ ]:
geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
geo_df = gpd.GeoDataFrame(df, geometry=geometry)

In [ ]:
geo_df.set_crs(germany_gdf.crs, inplace=True)

In [ ]:
fig, ax = plt.subplots()
germany_gdf.plot(ax=ax, color='lightgrey')

geo_df.plot(ax=ax, marker='o', color='red', markersize=5)

plt.show()

## Convert datetimes (column time)

In [ ]:
df["timestamps"] = pd.to_datetime(df["time"])
df["epoch_time"] = (df['timestamps'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
df = df.drop(columns=["timestamps"])
print("DataFrame with Epoch Time:")
df

In [ ]:
df["windspeed"] = np.sqrt(df['u10']**2 + df['v10']**2)

In [ ]:
df.columns

- **longitude, latitude**: Geografische Längen- und Breitengrade der Messpunkte oder Prognosepunkte.

- **forecast_origin**: Datum und Uhrzeit, wann die Wettervorhersage erstellt wurde.

- **time**: Datum und Uhrzeit, für die die Wettervorhersage gilt.

- **cdir**: Oft eine Abkürzung für „Cloud Direction“ oder „Convection Direction“, die die Windrichtung in bestimmten Höhen angibt.

- **z**: Normalerweise die geopotentielle Höhe, die die Höhe über dem Meeresspiegel in Bezug auf das geopotentielle Niveau anstelle des tatsächlichen Abstands angibt.

- **msl**: "Mean Sea Level Pressure", also der mittlere Luftdruck auf Meereshöhe.

- **blh**: "Boundary Layer Height", die Höhe der atmosphärischen Grenzschicht, was wichtig für viele Prozesse in der Meteorologie ist.

- **tcc**: "Total Cloud Cover", der gesamte Wolkenbedeckungsgrad, ausgedrückt als Prozentsatz oder Bruchteil.

- **u10, v10**: Windkomponenten (u und v) in 10 Metern Höhe. „u“ ist die Ost-West-Komponente und „v“ die Nord-Süd-Komponente.

- **t2m**: "Temperature at 2 meters", die Lufttemperatur in 2 Metern Höhe über dem Boden.

- **ssr**: "Surface Solar Radiation", die auf die Erdoberfläche eingestrahlte Sonnenenergie.

- **tsr**: "Top of Atmosphere Solar Radiation", die am oberen Rand der Atmosphäre eingestrahlte Sonnenenergie.

- **sund**: Wahrscheinlich eine Abkürzung für die Anzahl der Sonnenstunden oder Sonnendauer.

- **tp**: "Total Precipitation", die Gesamtniederschlagsmenge über einen bestimmten Zeitraum.

- **fsr**: Oft steht dies für "Forecast Solar Radiation", also die vorhergesagte Sonneneinstrahlung.

- **u100, v100**: Ähnlich wie u10 und v10, aber für Windkomponenten in 100 Metern Höhe.


## Lets plot the windspeed throughout the year

In [ ]:
import plotly.graph_objects as go


In [ ]:
fig = go.Figure()
x_axis = df["time"]

fig.add_trace(go.Scatter(x=x_axis, y=df.windspeed,
                    mode='lines',
                    name='windspeed'))
fig.add_trace(go.Scatter(x=x_axis, y=df.ssr,
                    mode='lines',
                    name='solar'))

fig.show()





## Nichts lineares 
- Mehr Wind muss nicht mehr Energie heissen - Turbinen hab kapa
- Solar ebenso - ab 25 Grad (?) Limit erreicht, danach decrease glaub

# Unsere Aufgabe
Predicte wind und solar für die nächste Stunde anhand
- Variablen suchen wir aus ? Wir haben aber den jetzigen Stand gegeben ?
    - Sprich Uhrzeit, jetzigen Wind, wo wir uns befinden, Monat, wie Wind hatten wir davor ?
- Dürfen wir selber Daten anreichern ? Wetter daten w.r.t Sonne, re

----------

In [5]:
df_prices = pd.read_csv('data/' + files[1], sep=';')
df_realized_supply = pd.read_csv('data/' + files[2], sep=';')
df_installed_cp = pd.read_csv('data/' + files[3], sep=';')
df_realized_demand = pd.read_csv('data/' + files[4], sep=';')

In [ ]:
df_prices

In [6]:
df_realized_supply

,Date from,Date to,Biomass [MW],Hydro Power [MW],Wind Offshore [MW],Wind Onshore [MW],Photovoltaic [MW],Other Renewable [MW],Nuclear Power [MW],Lignite [MW],Coal [MW],Natural Gas [MW],Pumped Storage [MW],Other Conventional [MW]
0,01.01.19 00:00,01.01.19 00:15,"1.199,00",401,"794,25","4.839,25",0,"31,5","2.250,50","1.733,00","836,25","850,5","87,5","431,75"
1,01.01.19 00:15,01.01.19 00:30,"1.203,00","394,75","793,5","5.030,75",0,"31,5","2.263,25","1.587,75","729,5","821,5","99,25","429,75"
2,01.01.19 00:30,01.01.19 00:45,"1.204,50","381,25","791,75","5.213,50",0,"31,5","2.246,00","1.555,25","648,5","804,75",44,"431,25"
3,01.01.19 00:45,01.01.19 01:00,"1.208,75",381,"754,5","5.308,25",0,"31,5","2.241,75","1.459,00","597,5","796,5",41,"431,5"
4,01.01.19 01:00,01.01.19 01:15,"1.204,50","388,25","715,5","5.398,50",0,"31,5","2.224,50","1.367,75","610,5","765,5",21,"429,25"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140251,31.12.22 22:45,31.12.22 23:00,"1.118,75","308,75","874,75","7.660,50",0,"30,5","759,75","940,5","551,75",394,29,"304,25"
140252,31.12.22 23:00,31.12.22 23:15,"1.111,25","317,75","842,5","7.390,25",0,"30,5","722,25","953,75","549,75","385,5",251,"305,25"
140253,31.12.22 23:15,31.12.22 23:30,"1.112,25",310,"852,25","7.356,00",0,"30,5","684,5","951,25","552,25","387,25","63,25","307,75"
140254,31.12.22 23:30,31.12.22 23:45,"1.109,25","311,5","842,25","7.301,75",0,"30,5","648,25","953,25",551,"391,5","15,25",307


In [ ]:
df_realized_demand

# Kalmann Filter

## Initial Estimate

$$\hat{\mathbf{x}}_{0,0}, \quad \mathbf{P}_{0,0}$$

### Extrapolate ("Predict")

1. Extrapolate the state:
   
   $$ \hat{\mathbf{x}}_{n+1,n} = \mathbf{F} \hat{\mathbf{x}}_{n,n} + \mathbf{G} \mathbf{u}_n $$
   

2. Extrapolate uncertainty:
   
   $$\mathbf{P}_{n+1,n} = \mathbf{F} \mathbf{P}_{n,n} \mathbf{F}^T + \mathbf{Q}$$

<br>
We extraplolate the state at time n and the uncertainty to the next time step. Our guesses can be updated with the measurement that takes place later on.
<br>

## Measurement Update ("Correct")
After we have the measurement at time n+1, we can update our estimates with the measurement.

1. Compute the Kalman Gain:

   $$\mathbf{K}_n = \mathbf{P}_{n,n-1} \mathbf{H}^T (\mathbf{H} \mathbf{P}_{n,n-1} \mathbf{H}^T + \mathbf{R}_n)^{-1}$$


2. Update estimate with measurement:
   
$$   \hat{\mathbf{x}}_{n,n} = \hat{\mathbf{x}}_{n,n-1} + \mathbf{K}_n (\mathbf{z}_n - \mathbf{H} \hat{\mathbf{x}}_{n,n-1})$$
   

3. Update the estimate uncertainty:

   $$\mathbf{P}_{n,n} = (\mathbf{I} - \mathbf{K}_n \mathbf{H}) \mathbf{P}_{n,n-1} (\mathbf{I} - \mathbf{K}_n \mathbf{H})^T + \mathbf{K}_n \mathbf{R}_n \mathbf{K}_n^T$$ 
   bzw.
   
   $$ \mathbf{P}_{n,n} = (\mathbf{I} - \mathbf{K}_n \mathbf{H}) \mathbf{P}_{n,n-1}$$
   




The Kalman gain basically makes nothing but variance in estimate divided by variance in estimate + variance in measurement. If the Kalman gain is close to zero, this means that the measurement uncertainty is high and the estimate uncertainty is low. So we gice more weight to the estimate and only small weight to the measurement itself. For the opposite case, we trust the measurement and give it hence more weight.

After having the Kalman gain, we can update our estimate with the measurement. We can also update the uncertainty of our estimate, as we had extraploated the uncertainty in the first step which we can now update. The reason for the equaiton to look like that is a long derivation. WHat we basically do is we know our estimate update. Because of linearity, we can plug in the variances instead of the estimate itself. After that, we want to minimize the variance of our estimate (kinda equals minimizing the MSE loss, having less variance means we are more certain about our estimate). Setting that equation to 0 and doing fancy stuff, we get the equation above.

After doing all these steps, we repeat the process. Our new estimates become our prior estimates for the next time step. There, we again extrapolate the state and uncertainty and update our estimates with the measurement. 

# Problem
- Ich hab keine Ahnunng wie ich F machen soll, geschweige denn Q
- Wie sieht das lineare Modell aus für das predicten des States ??
- Wie sieht mein State überhaupt aus ? hab ich u100,v100, wind on+offshore ? theoretisch hab ich on+offshore ja nicht, das ist das was ich ja durch das lineare modell durch u100 und v100 rausbekomme

## Regressionsanlayse ??

$$ Power (W) = \frac{1}{2} \times ρ \times A \times v^3 $$




Das trifft es doch gut, weil wir wie gesagt on und offshore so gesehen ja nie messen, das aber dadurch rauskregen könnten

- Power = Watts
- ρ (rho, a Greek letter) = density of the air in kg/m3
- A = cross-sectional area of the wind in m2
- v = velocity of the wind in m/s>

### Was genau sind meine state variables ? 